#Quelques idées d'optimisation d'un code numérique écrit en Python

Dans ce tutoriel, nous allons utiliser différents outils permettant d'optimiser du code numérique écrit en Python. Parmi, les outils proposés, nous nous intéresserons à

- Cython
- Pythran
- Numba

sur deux exemples simples. Le premier sera le calcul de $\pi$ où il y a une boucle à optimiser qui ne fait intervenir que des scalaires. Le deuxième cas est la résolution de l'équation de la chaleur en 2D par un schéma aux différences finies couplé à un schéma d'Euler explicite pour l'intégration en temps.

Nous regarderons pour chaque exemple les temps de calcul afin de comparer chacune des méthodes.

##Calcul de $\pi$

Il existe différentes manières de calculer une approximation du nombre $\pi$. Nous choisirons celle qui consiste à calculer l'intégrale suivante

$$
\pi = \int_0^1f(x)dx \; \text{avec} \; f(x)=\frac{4}{1+x^2}.
$$

Pour ce faire, on approche l'intégrale en utilisant une méthode des rectangles

$$
\pi \approx \frac{1}{n}\sum_{i=0}^{n-1}f(x_i), \; \text{avec} \; x_i=\frac{i+0.5}{n} \; \text{pour} \; i=0,\cdots,n-1.
$$

Voici une écriture possible en Python

In [ ]:
def f(x):
    return 4./(1 + x**2)

def calculPi(n):
    h, pi = 1./n, 0.
    for i in xrange(n):
        pi += f(h*(i+.5))
    return h*pi

### Utilisation de Cython

Cython fonctionne de la façon suivante

- On écrit un fichier .pyx indiquant comment s'opère l'interfaçage entre les objets Python et du C.
- On utilise la commande cython pour créer l'interface .c écrite en API Python/C.
- On compile ce fichier .c en .o.
- On crée la librairie .so associée.
- On peut maintenant importer cette librairie dans Python et utiliser les fonctions ou classes ainsi créées.

Vous pouvez effectuer toutes ces étapes 

- utilisant `cython` manuellement,
- écrivant un fichier `setup.py` utilisant `distutils`,
- utilisant `pyximport`,
- utilisant Sage.

Etant donné que nous allons rester dans le notebook de ipython, nous utiliserons le module externe `Cython` qui permet d'écrire le fichier pyx dans une cellule de code. En voici un exemple

In [ ]:
%load_ext Cython

In [ ]:
%%cython

def hello():
    print("hello")

In [ ]:
hello()

Pour optimiser une partie d'un code numérique écrit en Python en utilisant Cython, vous avez 4 étapes 

- Faire un copier coller de la fonction à optimiser.
- Typer en C les variables de la fonction.
- Ajouter des directives si c'est nécessaire.
- Améliorer le code pour qu'il soit le plus possible en C.

Voici quelques directives importantes

- *boundscheck* 

    Si `False`, Cython assume que les indices demandés existent (`True` par défaut).


- *wraparound*

    Si `False`, Cython ne vérifie pas si l'indice est négatif (`True` par défaut).


- *cdivision* 

    Si `True`, Cython fait une division en C (`False` par défaut).
    
On peut les utiliser de différentes manières.

- En ligne de commande

        cython -X boundscheck=True monfichier.pyx
  
  
- A toutes les fonctions du fichier `.pyx`

        #cython: boundscheck=True
        
        
- Uniquement à une fonction

        @cython.boundscheck(True)
        def mafonction(...):

#### Effectuez ces étapes sur la fonction calculPi

In [ ]:
%%cython
def f(x):
    return 4./(1 + x**2)

def calculPiCython(n):
    h, pi = 1./n, 0.
    for i in xrange(n):
        pi += f(h*(i+.5))
    return h*pi

In [ ]:
%timeit calculPi(100000)

In [ ]:
%timeit calculPiCython(100000)

###Utilisation de Pythran et de Numba

Contrairement à Cython, Pythran et Numba agissent directement sur le fichier Python. Nous allons indiquer par un commentaire pour Pythran et par un décorateur pour Numba, que nous voulons générer une version optimisée de la fonction ainsi décorée.

La grande différence entre Pythran et Numba est que Pythran génère le code optimisé avant l'exécution du programme alors que Numba génère le code optimisé au runtime.

#### Exemple Pythran

Comme dit précédemment, il suffit de mettre un commentaire sur la fonction à optimiser pour que Pythran essaie de le faire. Ce commentaire commence toujours par la même syntaxe

`#pythran export`

Il est ensuite suivi du nom de la fonction et du type des paramètres en argument. Le fait de spécifier les types est primordial pour que Pythran puisse faire de l'inférence de types dans la fonction et donc comprendre tous les objets que nous manipulons.

Les types définis dans Pythran sont

In [ ]:
argument_type = basic_type
                          | (argument_type+)    # this is a tuple
                          | argument_type list  # this is a list
                          | argument_type set   # this is a set
                          | argument_type []+   # this is a ndarray
                          | argument_type [::]+ # this is a strided ndarray
                          | argument_type:argument_type dict    # this is a dictionary

basic_type = bool | int | long | float | str
       | uint8 | uint16 | uint32 | uint64
       | int8 | int16 | int32 | int64
       | float32 | float64
       | complex64 | complex128

Commençons par un exemple simple

In [ ]:
%load_ext pythranmagic

In [ ]:
%%pythran
#pythran export helloPythran()
def helloPythran():
    return 'hello Pythran'

In [ ]:
helloPythran()

Appliquez Pythran sur la fonction qui calcul $\pi$.

In [ ]:
%%pythran
#compléter

In [ ]:
print calculPiPythran(100000)
%timeit calculPiPythran(100000)

####Exemple Numba

Numba s'utilise un peu de la même manière que Pythran pour choisir la fonction à optimiser: il suffit de mettre un décorateur représenté en Python par le symbole `@`.

On importe donc le module Numba et on ajoute au dessus de la fonction à optimiser

`@numba.jit`

Vous pouvez également définir des arguments dans `jit` permettant de spécifier les arguments de la fonction mais Numba essaie de les découvrir lui-même au runtime.

Le fait que la compilation se fasse au runtime, la première fois que vous appelez votre fonction, celle-ci est générée puis compilée. Si vous faites des tests de performances, il faut donc bien faire attention à ce que vous voulez. 

In [ ]:
import numba
@numba.autojit
def hello():
    return 'hello'

In [ ]:
hello()

Appliquez Numba sur la fonction qui calcul $\pi$.

In [ ]:
@numba.jit
#compléter

In [ ]:
calculPiNumba(1)
%timeit calculPiNumba(100000)

##Equation de la chaleur

Nous souhaitons maintenant résoudre l'équation de la chaleur 2D en utilisant un schéma d'**Euler explicite** en temps et un schéma aux différences dinies en espace.

$$
\left\{
\begin{array}{l}
\frac{\partial u}{\partial t}-\Delta u = 0 \; \text{sur} \; \Omega, \\\\
u = 0 \; \text{sur} \; \partial \Omega.
\end{array}
\right.
$$

en prenant la solution initiale suivante

$$
u_0(x, y) = 100e^{-100((x-0.5)^2 + (y -0.5)^2))}.
$$

On prendra comme domaine $\Omega=[0, 1]\times[0,1]$, comme pas d'espace $h_x=0.01$, $h_y=0.01$ et enfin comme pas de temps $dt=h_x^2/4$.

####Mise en place des conditions de bords

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt 
def setDirichlet(y):
    y[0, :] = 0
    y[-1, :] = 0
    y[:, 0] = 0
    y[:, -1] = 0

#### Calcul du terme $\Delta u$

On calcule de Laplacien par la formule

$$
\Delta u(x_i, y_j) \approx \frac{u_{i-1,j}-2u_{i,j}+u_{i+1,j}}{h_x^2}+\frac{u_{i,j-1}-2u_{i,j}+u_{i,j+1}}{h_y^2}.
$$

In [ ]:
def laplacian2D(x, h):
    """
    Produit matrice-vecteur du Laplacien sans assemblage de la matrice

    """
    cx = -1./h[0]**2
    cy = -1./h[1]**2
    c2 = 2./h[0]**2 + 2./h[1]**2 
    y = np.empty(x.shape)
    y[1:-1, 1:-1] = (c2*x[1:-1, 1:-1] 
                     + cy*(x[2:, 1:-1] + x[:-2, 1:-1])
                     + cx*(x[1:-1, 2:] + x[1:-1, :-2]))
    
    return y

#### Méthode d'Euler explicite dans le cas général

Etant donné un pas de temps $\Delta t$ et une suite d'instants $(t_n = t_0 + n\Delta t)_{n\in \mathbb{N}}$, la méthode d'Euler explicite associée à l'équation différentielle

$$
u'(t) = f(t, u(t))
$$

s'écrit

$$
u_{n+1} = u_n + \Delta t f(t_n, u_n)
$$

In [ ]:
def euler(u, dt, f, fargs=()):
    """
    Euler explicite pour un système 

    u'(t) = f(t, u(t))
    
    Parameters :
    ------------

    u: solution à l'instant n
    dt:  pas de temps du schéma
    f: fonction second membre
    fargs: paramètres optionnels de la fonction autre que u

    """
    return u + dt*f(u, *fargs)

#### Initialisation de la solution

On rappelle que la solution initiale est de la forme

$$
u_0(x, y) = 100e^{-100((x-0.5)^2 + (y -0.5)^2))}.
$$

In [ ]:
def initSol(nx, ny, Lx=[0., 1.], Ly=[0., 1.]):
    """
    Initialisation d'une fonction pour tester le schéma en temps

    """
    x = np.linspace(Lx[0], Lx[1], nx)
    y = np.linspace(Ly[0], Ly[1], ny)
    
    x = x[np.newaxis, :]
    y = y[:, np.newaxis]

    return 100.*np.exp(-100.*((x - .5)**2 + (y - .5)**2))

#### Calcul de la solution de l'équation de la chaleur

On choisit les paramètres suivants pour l'ensemble de nos simulations.

In [ ]:
nx, ny = 101, 101
h = [1./(nx + 1), 1./(ny + 1)]

dt = np.min(h)**2/4.
nite = 100

u = initSol(nx, ny)

On définit ici la fonction qui va permettre d'animer notre solution via matplotlib.

In [ ]:
def init():
    image.set_data(u)    
    time_text.set_text('')
    return image, time_text

def animate(i):
    u[:, :] = euler(u, -dt, laplacian2D, fargs=(h,))
    setDirichlet(u)
    image.set_data(u)

    time_text.set_text(time_template%(i*dt))
    return image, time_text

In [ ]:
from JSAnimation.IPython_display import display_animation
import matplotlib.animation as animation

# Creation de la figure
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, autoscale_on=True)

# Creation des différents elements constituants la figure
image = ax.imshow(u)
time_template = 'time = %es'
time_text = ax.text(0.05, 0.05, 'test', transform=ax.transAxes, color="white", fontsize=18)

anim = animation.FuncAnimation(fig, animate, xrange(nite),
    interval=25, blit=True, init_func=init)

In [ ]:
from JSAnimation.IPython_display import display_animation
display_animation(anim, default_mode='once')

Nous allons maintenant essayer d'optimiser la fonction `laplacian2D` en utilisant Cython, Pythran et Numba.

####Version Cython

Nous manipulons ici des tableaux Numpy. Dans Cython, nous pouvons accéder à ces tableaux en utilisant une *memory view*. La syntaxe est simple: il suffit de donner le type et les dimensions du tableau avec éventuellement son stockage C ou Fortran.

In [ ]:
cdef int [:, :, ::1] mvc

#memory view d'un tableau numpy stocké en C
mvc = np.zeros((10, 10, 10), dtype=np.int32)

cdef int [::1, :, :] mvF

#memory view d'un tableau numpy stocké en Fortran
mvF = np.zeros((10, 10, 10), dtype=np.int32, order='F')

Ecrivez la version Cython de `laplacian2D`. On rappelle les 4 grandes étapes

- Faire un copier coller de la fonction à optimiser.
- Typer en C les variables de la fonction.
- Ajouter des directives si c'est nécessaire.
- Améliorer le code pour qu'il soit le plus possible en C.

In [ ]:
%%cython -a 
import numpy as pnp
import cython
    
#compléter

In [ ]:
%timeit laplacian2D(np.zeros((1000,1000)), np.array([1.,1.]))

In [ ]:
%timeit laplacian2DCython(np.zeros((1000,1000)), np.array([1., 1.]))

Ecrivez la version Numba de `laplacian2D`.

In [ ]:
@numba.jit
#compléter

In [ ]:
laplacian2DNumba(np.zeros((10,10)), np.array([1., 1.]))
%timeit laplacian2DNumba(np.zeros((1000,1000)), np.array([1., 1.]))

Ecrivez la version Pythran de `laplacian2D`.

In [ ]:
%%pythran
#compléter

In [ ]:
import numpy as np
%timeit laplacian2DPythran(np.zeros((1000,1000)), np.array([1., 1.]))

In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open("styles/custom.css", "r").read()
    return HTML(styles)
css_styling()